In [1]:
# imports
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_replace, monotonically_increasing_id
from pyspark.sql.types import IntegerType, DecimalType, StringType

In [2]:
# iniciar spark
conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1228769e-ecd0-47c6-b74d-f21478ea8022;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 413ms :: artifacts dl 22ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [3]:
# Variáveis S3
bucket_bruto = 'mario-sprint3-heleno-brutos'
bucket_tratado = 'mario-sprint3-heleno-tratados'

In [4]:
def upload_df_s3(df, bucket_name, file_name, type_file='csv'):
    if type_file == 'csv':
        df.write \
        .option("header", True) \
        .option("delimiter",";") \
        .mode('overwrite') \
        .csv(f's3a://{bucket_name}/{file_name}')
    elif type_file == 'json':
        df.write \
       .mode('overwrite') \
       .json(f's3a://{bucket_name}/{file_name}')
    else:
        print('Extensão de arquivo inválida')

In [5]:
def file_to_dataframe(bucket_name, file_name, delimiter=';', file_format='csv'):
    if file_format == 'csv':
        return spark.read.option('delimiter', delimiter).option('header', 'true').csv(f's3a://{bucket_name}/{file_name}')
    elif file_format == 'json':
        return spark.read.option("multiline","true").json(f's3a://{bucket_name}/{file_name}')
    else:
        print('Formato inválido')

In [6]:
# load s3 file
file_name = 'Value_of_Production_E_All_Data_(Normalized).csv'

df = file_to_dataframe(bucket_bruto, file_name, ',')

22/06/07 23:19:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
df_filtrado = df.filter('Area IN ("Argentina", "Brazil", "China", "Germany")') \
    .filter('Item IN ("Meat, cattle", "Meat, chicken", "Meat, pig", "Eggs, hen, in shell")') \
    .filter('Element = "Gross Production Value (current thousand US$)"')

In [8]:
df_filtrado.count()

458

In [9]:
# Configurações do arquivo a ser gerado no S3
file_name = 'meats_1991_2019'
file_extension = 'csv'

# Montando o dataframe com o nome dos campos como o da tabela
_df = df_filtrado.select(
    col('Area').alias('Pais'),
    col('Item').alias('Produto'),
    col('Year').cast(IntegerType()).alias('Ano'),
    col('Value').cast(DecimalType(17, 2)).alias('Valor')
)

upload_df_s3(_df, bucket_tratado, file_name, file_extension)

22/06/07 23:20:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 23:20:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 23:20:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 23:20:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/07 23:20:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
